<a href="https://colab.research.google.com/github/Shashank014cyber/Expenses-Tracker-WebApp/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas openpyxl langchain faiss-cpu sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 69.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving 2025batch_placements (1).xlsx to 2025batch_placements (1) (1).xlsx
Saving 2025batch_placements.xlsx to 2025batch_placements (2).xlsx
Saving 2024_data(Autosaved) (1).xlsx to 2024_data(Autosaved) (1) (1).xlsx
Saving 2024_data(Autosaved).xlsx to 2024_data(Autosaved) (2).xlsx


In [ ]:
file_paths = list(uploaded.keys())
dfs = [pd.read_excel(fp, header=1) for fp in file_paths]


In [ ]:
combined_df = pd.concat(dfs, ignore_index=True)
print("Detected columns in combined_df:")
print(combined_df.columns.tolist())

Detected columns in combined_df:
['Department', 'Name of the outgoing student placed', 'Male or Female or Others', 'Roll No. of the Student', 'Programme Graduated from', 'Year of Placement', 'Organization in which placed with contact details', 'Region of employment\n(City Name )', 'Self Placed or Campus Placement or Entrepreneurship', 'Package committed per annum in INR', 'Link to the Soft Copy of Offer Letter', 'Link to the Soft Copy of Appointment Letter', '2023-24', '1/20/FET/BCS/001', 'Aastha Singh', 'CSE', 'Higher Studies', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'SET', 'Conv', 'PGDAI', 'CDAC, Noida', 'Unnamed: 12', 'Unnamed: 13', 'Admission Confirmation']


In [ ]:
def get_column(df, name_keyword):
    for col in df.columns:
        if name_keyword.lower() in col.lower():
            return col
    return None

In [ ]:
combined_df = combined_df.dropna(thresh=4)
combined_df = combined_df.reset_index(drop=True)


In [ ]:
name_col = get_column(combined_df, "student placed") or get_column(combined_df, "student name")
dept_col = get_column(combined_df, "department")
org_col = get_column(combined_df, "organization") or get_column(combined_df, "employer")
year_col = get_column(combined_df, "year")
city_col = get_column(combined_df, "region") or get_column(combined_df, "place")
prog_col = get_column(combined_df, "programme") or get_column(combined_df, "program")
pkg_col = get_column(combined_df, "package") or get_column(combined_df, "pay")
type_col = get_column(combined_df, "self placed") or get_column(combined_df, "placement type")

print("\nMatched columns:")
print(f"Name: {name_col}, Department: {dept_col}, Organization: {org_col}")
print(f"Year: {year_col}, City: {city_col}, Program: {prog_col}, Package: {pkg_col}, Placement Type: {type_col}")



Matched columns:
Name: Name of the outgoing student placed, Department: Department, Organization: Organization in which placed with contact details
Year: Year of Placement, City: Region of employment
(City Name ), Program: Programme Graduated from, Package: Package committed per annum in INR, Placement Type: Self Placed or Campus Placement or Entrepreneurship


In [ ]:
print("\nMatched columns:")
print(f"Name: {name_col}, Department: {dept_col}, Organization: {org_col}")
print(f"Year: {year_col}, City: {city_col}, Program: {prog_col}, Package: {pkg_col}, Placement Type: {type_col}")



Matched columns:
Name: Name of the outgoing student placed, Department: Department, Organization: Organization in which placed with contact details
Year: Year of Placement, City: Region of employment
(City Name ), Program: Programme Graduated from, Package: Package committed per annum in INR, Placement Type: Self Placed or Campus Placement or Entrepreneurship


In [ ]:
docs = []
for _, row in combined_df.iterrows():
    try:
        name = str(row.get(name_col, "")).strip()
        dept = str(row.get(dept_col, "")).strip()
        org = str(row.get(org_col, "")).strip()
        year = str(row.get(year_col, "")).strip()
        city = str(row.get(city_col, "")).strip()
        prog = str(row.get(prog_col, "")).strip()
        pkg = str(row.get(pkg_col, "")).strip()
        placement_type = str(row.get(type_col, "Not specified")).strip()

        if name and dept and org:
            sentence = (
                f"{name} from the {dept} department got placed at {org} in {city} in {year} "
                f"after completing {prog}, with a package of {pkg} INR per annum. "
                f"The placement type was: {placement_type}."
            )
            docs.append(sentence)
    except Exception as e:
        continue  # skip row if any problem

print(f"\nTotal placement records processed into sentences: {len(docs)}")



Total placement records processed into sentences: 286


In [ ]:
# ✅ DEBUG: Check if any placement sentences were created
print(f"\n✅ Total placement records processed into sentences: {len(docs)}")

# If no records were found, raise an error and stop
if len(docs) == 0:
    print("❌ ERROR: No valid placement data found. Please check the column matching.")

    # Print matched columns for inspection
    print("\n🔍 Matched column names:")
    print(f"name_col: {name_col}")
    print(f"dept_col: {dept_col}")
    print(f"org_col: {org_col}")
    print(f"year_col: {year_col}")
    print(f"city_col: {city_col}")
    print(f"prog_col: {prog_col}")
    print(f"pkg_col: {pkg_col}")
    print(f"type_col: {type_col}")

    # Print first few rows to inspect data structure
    print("\n🧪 Sample row:")
    print(combined_df.head(1).T)  # transpose for readability

    raise ValueError("No data to embed. Stopping execution.")



✅ Total placement records processed into sentences: 286


In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(docs)

In [ ]:
dimension = embeddings[0].shape[0]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))

In [ ]:
# Step 8: Create FAISS index
dimension = embeddings[0].shape[0]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))


In [ ]:
# Step 9: Define chatbot function
def ask_bot(question, top_k=3):
    question_vec = model.encode([question])
    distances, indices = index.search(np.array(question_vec), top_k)
    answers = [docs[i] for i in indices[0]]
    return "\n\n".join(answers)


In [ ]:
# Step 10: Ask a sample question
question = "Which students got placed in Infosys?"
print("Question:", question)
print("\nAnswer:\n", ask_bot(question))


Question: Which students got placed in Infosys?

Answer:
 ADITYA JAISWAL from the CSE department got placed at Artech Infosystems in Noida in 2025.0 after completing B.Tech, with a package of 4.68 INR per annum. The placement type was: Campus Placement.

ADITYA JAISWAL from the CSE department got placed at Artech Infosystems in Noida in 2025.0 after completing B.Tech, with a package of 4.68 INR per annum. The placement type was: Campus Placement.

SHRIYA GODARA from the CSE department got placed at Infogain in Noida in 2024.0 after completing B.Tech, with a package of 4.34 INR per annum. The placement type was: Campus Placement.
